In [1]:
import pandas as pd #数据分析
import numpy as np #科学计算

import visuals as rs
from sklearn import tree
from sklearn.metrics import r2_score
from sklearn import cross_validation
from IPython.display import display # 使得我们可以对DataFrame使用display()函数
from sklearn import cluster
from sklearn import decomposition

import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

/Users/mahui/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/mahui/anaconda/lib/python2.7/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


# 一、了解目标数据集和学习目标，合并train和test便于清洗

In [2]:
#查看数据集大小
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
data_name = train.columns
data_name


train.shape,test.shape

((1460, 81), (1459, 80))

In [3]:
#确定模型要学习的目标，
target='SalePrice'
IDcol = 'Id'

In [4]:
#添加一列，合成一个总的data，便于清洗
train['source']= 'train'
test['source'] = 'test'
data=pd.concat([train, test],ignore_index=True)
data.shape

(2919, 82)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 82 columns):
1stFlrSF         2919 non-null int64
2ndFlrSF         2919 non-null int64
3SsnPorch        2919 non-null int64
Alley            198 non-null object
BedroomAbvGr     2919 non-null int64
BldgType         2919 non-null object
BsmtCond         2837 non-null object
BsmtExposure     2837 non-null object
BsmtFinSF1       2918 non-null float64
BsmtFinSF2       2918 non-null float64
BsmtFinType1     2840 non-null object
BsmtFinType2     2839 non-null object
BsmtFullBath     2917 non-null float64
BsmtHalfBath     2917 non-null float64
BsmtQual         2838 non-null object
BsmtUnfSF        2918 non-null float64
CentralAir       2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
Electrical       2918 non-null object
EnclosedPorch    2919 non-null int64
ExterCond        2919 non-null object
ExterQual        2919 non-null object
Exterior1st      291

# 二、处理缺失值<br><br>

有两种方式：在原数据集上增删、新建一个数据表往上加，感觉后面一种方便一些，现在用的是前面这种<br>
- 1、查看缺失值及缺失值的分布情况


In [6]:
#查看缺省值
data_miss = data.isnull().apply(lambda x: x.sum(), axis=0)

data_miss = pd.DataFrame({'Index':data_miss.index,'Values':data_miss.values})
var=list()
for i,j in data_miss.iterrows():
    if data_miss.iloc[i,1] > 0:
        var.append(data_miss.iloc[i,0])

print '共%s个特征有缺失值' % len(var)

共35个特征有缺失值


In [7]:
for v in var:
    print '\n\n%s这一列特征的不同取值，共有%s个缺失值" \n'% (v,data[v].isnull().sum())
    print data[v].value_counts()
    print '缺失值：%s 个' %data[v].isnull().sum()



Alley这一列特征的不同取值，共有2721个缺失值" 

Grvl    120
Pave     78
Name: Alley, dtype: int64
缺失值：2721 个


BsmtCond这一列特征的不同取值，共有82个缺失值" 

TA    2606
Gd     122
Fa     104
Po       5
Name: BsmtCond, dtype: int64
缺失值：82 个


BsmtExposure这一列特征的不同取值，共有82个缺失值" 

No    1904
Av     418
Gd     276
Mn     239
Name: BsmtExposure, dtype: int64
缺失值：82 个


BsmtFinSF1这一列特征的不同取值，共有1个缺失值" 

0.0      929
24.0      27
16.0      14
300.0      9
384.0      8
600.0      8
288.0      8
20.0       8
375.0      7
936.0      7
700.0      7
500.0      7
456.0      7
624.0      7
602.0      7
360.0      7
468.0      6
560.0      6
544.0      6
625.0      6
120.0      6
662.0      6
528.0      6
547.0      6
588.0      6
276.0      6
553.0      6
368.0      6
504.0      6
312.0      6
        ... 
873.0      1
867.0      1
863.0      1
862.0      1
859.0      1
854.0      1
853.0      1
852.0      1
896.0      1
899.0      1
950.0      1
900.0      1
949.0      1
946.0      1
943.0      1
939.0      1
937.0      1
935.0      

- 2、对于有说明的字符型，用新特征代替Nan

In [8]:
#对所有有说明的字符型处理，用一个新特征代表缺失


#门口铺设，Alley，NaN 代表 No alley access，替换为 NoAlley
#砌体类型，MasVnrType，NaN代表无砌体
#地下室高度、状况、光照、评级、第二地下室评级，BsmtQual、BsmtCond、BsmtExposure、BsmtFinType1、BsmtFinType2，NaN 代表 No Basement，替换为 NoBasement
#游泳池质量，PoolQC，NA 代表 No Pool
#栅栏质量，Fence，NA 代表 No Fence
#杂项功能，MiscFeature，NA 代表 None
#车库类型 、建成时间 、装饰、质量、条件，GarageType、GarageFinish、GarageCars、GarageArea、GarageQual NA代表No Garage



data.loc[ (data['Alley'].isnull()), 'Alley' ] = None

data.loc[ (data['MasVnrType'].isnull()), 'MasVnrType' ] = None

data.loc[ (data['BsmtQual'].isnull()), 'BsmtQual' ] = None

data.loc[ (data['BsmtCond'].isnull()), 'BsmtCond' ] = None

data.loc[ (data['BsmtExposure'].isnull()), 'BsmtExposure' ] = None

data.loc[ (data['BsmtFinType1'].isnull()), 'BsmtFinType1' ] = None

data.loc[ (data['BsmtFinType2'].isnull()), 'BsmtFinType2' ] = None

data.loc[ (data['PoolQC'].isnull()), 'PoolQC' ] = None

data.loc[ (data['Fence'].isnull()), 'Fence' ] = None

data.loc[ (data['MiscFeature'].isnull()), 'MiscFeature' ] = None

data.loc[ (data['GarageType'].isnull()), 'GarageType' ] = None

data.loc[ (data['GarageQual'].isnull()), 'GarageQual' ] = None

data.loc[ (data['GarageCond'].isnull()), 'GarageCond' ] = None

data.loc[ (data['GarageFinish'].isnull()), 'GarageFinish' ] = None

data.loc[ (data['FireplaceQu'].isnull()), 'FireplaceQu' ] = None

In [9]:
#查看缺省值
data_miss = data.isnull().apply(lambda x: x.sum(), axis=0)

data_miss = pd.DataFrame({'Index':data_miss.index,'Values':data_miss.values})
var=list()
for i,j in data_miss.iterrows():
    if data_miss.iloc[i,1] > 0:
        var.append(data_miss.iloc[i,0])

print '共%s个特征有缺失值' % len(var)

for v in var:
    print '\n\n%s这一列特征的不同取值，共有%s个缺失值" \n'% (v,data[v].isnull().sum())
    print data[v].value_counts()
    print '缺失值：%s 个' %data[v].isnull().sum()

共35个特征有缺失值


Alley这一列特征的不同取值，共有2721个缺失值" 

Grvl    120
Pave     78
Name: Alley, dtype: int64
缺失值：2721 个


BsmtCond这一列特征的不同取值，共有82个缺失值" 

TA    2606
Gd     122
Fa     104
Po       5
Name: BsmtCond, dtype: int64
缺失值：82 个


BsmtExposure这一列特征的不同取值，共有82个缺失值" 

No    1904
Av     418
Gd     276
Mn     239
Name: BsmtExposure, dtype: int64
缺失值：82 个


BsmtFinSF1这一列特征的不同取值，共有1个缺失值" 

0.0      929
24.0      27
16.0      14
300.0      9
384.0      8
600.0      8
288.0      8
20.0       8
375.0      7
936.0      7
700.0      7
500.0      7
456.0      7
624.0      7
602.0      7
360.0      7
468.0      6
560.0      6
544.0      6
625.0      6
120.0      6
662.0      6
528.0      6
547.0      6
588.0      6
276.0      6
553.0      6
368.0      6
504.0      6
312.0      6
        ... 
873.0      1
867.0      1
863.0      1
862.0      1
859.0      1
854.0      1
853.0      1
852.0      1
896.0      1
899.0      1
950.0      1
900.0      1
949.0      1
946.0      1
943.0      1
939.0      1
937.0      1


In [10]:
#通过neighborhood与LotFrontage，与相关性较高的特征组成新的特征，代替LotFrontage缺失项（此特征重要，且缺失较多，明显故意的嘛）

lot_frontage_by_neighborhood = data["LotFrontage"].groupby(data["Neighborhood"])

data["LotFrontage"] = data["LotFrontage"]   
for key, group in lot_frontage_by_neighborhood:
    idx = (data["Neighborhood"] == key) & (data["LotFrontage"].isnull())
    data.loc[idx, "LotFrontage"] = group.median()

In [11]:
#关于质量等的字符型特征，统一换成数值型
qual_dict = {None: 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5}
data["PoolQC"] = data["PoolQC"].map(qual_dict).astype(int)
data["ExterQual"] = data["ExterQual"].map(qual_dict).astype(int)
data["ExterCond"] = data["ExterCond"].map(qual_dict).astype(int)
data["BsmtQual"] = data["BsmtQual"].map(qual_dict).astype(int)
data["BsmtCond"] = data["BsmtCond"].map(qual_dict).astype(int)
data["HeatingQC2"] = data["HeatingQC"].map(qual_dict).astype(int)
data["KitchenQual"] = data["KitchenQual"].map(qual_dict).astype(int)
data["FireplaceQu"] = data["FireplaceQu"].map(qual_dict).astype(int)
data["GarageQual"] = data["GarageQual"].map(qual_dict).astype(int)
data["GarageCond"] = data["GarageCond"].map(qual_dict).astype(int)
data["BsmtExposure"] = data["BsmtExposure"].map({None: 0, "No": 1, "Mn": 2, "Av": 3, "Gd": 4}).astype(int)


bsmt_fin_dict = {None: 0, "Unf": 1, "LwQ": 2, "Rec": 3, "BLQ": 4, "ALQ": 5, "GLQ": 6}
data["BsmtFinType1"] = data["BsmtFinType1"].map(bsmt_fin_dict).astype(int)
data["BsmtFinType2"] = data["BsmtFinType2"].map(bsmt_fin_dict).astype(int)



data["Functional"] = data["Functional"].map(
{None: 0, "Sal": 1, "Sev": 2, "Maj2": 3, "Maj1": 4, 
 "Mod": 5, "Min2": 6, "Min1": 7, "Typ": 8}).astype(int)

data["GarageFinish"] = data["GarageFinish"].map(
{None: 0, "Unf": 1, "RFn": 2, "Fin": 3}).astype(int)

data["Fence"] = data["Fence"].map(
{None: 0, "MnWw": 1, "GdWo": 2, "MnPrv": 3, "GdPrv": 4}).astype(int)

In [12]:
#对于字符型的标签进行自动编码

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

def factorize(df, factor_df, column, fill_na=None):
    factor_df[column] = df[column]
    if fill_na is not None:
        factor_df[column].fillna(fill_na, inplace=True)
    le.fit(factor_df[column].unique())
    factor_df[column] = le.transform(factor_df[column])
    return factor_df
data["Neighborhood2"] = data["Neighborhood"]
data["SaleCondition2"] = data["SaleCondition"]


data = factorize(data, data, "MSSubClass")
data = factorize(data, data, "MSZoning", "RL")
data = factorize(data, data, "LotConfig")
data = factorize(data, data, "Neighborhood2")
data = factorize(data, data, "Condition1")
data = factorize(data, data, "BldgType")
data = factorize(data, data, "HouseStyle")
data = factorize(data, data, "RoofStyle")
data = factorize(data, data, "Exterior1st", "Other")
data = factorize(data, data, "Exterior2nd", "Other")
data = factorize(data, data, "MasVnrType", "None")
data = factorize(data, data, "Foundation")
data = factorize(data, data, "SaleType", "Oth")
data = factorize(data, data, "SaleCondition2")

In [13]:
#对一些出现概率较小特征进行处理


# IR2 and IR3 don't appear that often, so just make a distinction
# between regular and irregular.
data["IsRegularLotShape"] = (data["LotShape"] == "Reg") * 1

# Most properties are level; bin the other possibilities together
# as "not level".
data["IsLandLevel"] = (data["LandContour"] == "Lvl") * 1

# Most land slopes are gentle; treat the others as "not gentle".
data["IsLandSlopeGentle"] = (data["LandSlope"] == "Gtl") * 1

# Most properties use standard circuit breakers.
data["IsElectricalSBrkr"] = (data["Electrical"] == "SBrkr") * 1

# About 2/3rd have an attached garage.
data["IsGarageDetached"] = (data["GarageType"] == "Detchd") * 1

# Most have a paved drive. Treat dirt/gravel and partial pavement
# as "not paved".
data["IsPavedDrive"] = (data["PavedDrive"] == "Y") * 1

# The only interesting "misc. feature" is the presence of a shed.
data["HasShed"] = (data["MiscFeature"] == "Shed") * 1.  

# If YearRemodAdd != YearBuilt, then a remodeling took place at some point.
data["Remodeled"] = (data["YearRemodAdd"] != data["YearBuilt"]) * 1

# Did a remodeling happen in the year the house was sold?
data["RecentRemodel"] = (data["YearRemodAdd"] == data["YrSold"]) * 1

# Was this house sold in the year it was built?
data["VeryNewHouse"] = (data["YearBuilt"] == data["YrSold"]) * 1

data["Has2ndataloor"] = (data["2ndFlrSF"] == 0) * 1
data["HasMasVnr"] = (data["MasVnrArea"] == 0) * 1
data["HasWoodDeck"] = (data["WoodDeckSF"] == 0) * 1
data["HasOpenPorch"] = (data["OpenPorchSF"] == 0) * 1
data["HasEnclosedPorch"] = (data["EnclosedPorch"] == 0) * 1
data["Has3SsnPorch"] = (data["3SsnPorch"] == 0) * 1
data["HasScreenPorch"] = (data["ScreenPorch"] == 0) * 1

In [14]:
#将一些较多值的进行离散化，突出有意义的特征值



# Months with the largest number of deals may be significant.
data["HighSeason"] = data["MoSold"].replace( 
{1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 1, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0})

data["NewerDwelling"] = data["MSSubClass"].replace(
{20: 1, 30: 0, 40: 0, 45: 0,50: 0, 60: 1, 70: 0, 75: 0, 80: 0, 85: 0,
 90: 0, 120: 1, 150: 0, 160: 0, 180: 0, 190: 0})   
data.loc[data.Neighborhood == 'NridgHt', "Neighborhood_Good"] = 1
data.loc[data.Neighborhood == 'Crawfor', "Neighborhood_Good"] = 1
data.loc[data.Neighborhood == 'StoneBr', "Neighborhood_Good"] = 1
data.loc[data.Neighborhood == 'Somerst', "Neighborhood_Good"] = 1
data.loc[data.Neighborhood == 'NoRidge', "Neighborhood_Good"] = 1
data["Neighborhood_Good"].fillna(0, inplace=True)

data["SaleCondition_PriceDown"] = data.SaleCondition.replace(
{'Abnorml': 1, 'Alloca': 1, 'AdjLand': 1, 'Family': 1, 'Normal': 0, 'Partial': 0})

# House completed before sale or not
data["BoughtOffPlan"] = data.SaleCondition.replace(
{"Abnorml" : 0, "Alloca" : 0, "AdjLand" : 0, "Family" : 0, "Normal" : 0, "Partial" : 1})

data["BadHeating"] = data.HeatingQC.replace(
{'Ex': 0, 'Gd': 0, 'TA': 0, 'Fa': 1, 'Po': 1})

area_cols = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 
 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'LowQualFinSF', 'PoolArea' ]
data["TotalArea"] = data[area_cols].sum(axis=1)

data["TotalArea1st2nd"] = data["1stFlrSF"] + data["2ndFlrSF"]

data["Age"] = 2010 - data["YearBuilt"]
data["TimeSinceSold"] = 2010 - data["YrSold"]

data["SeasonSold"] = data["MoSold"].map({12:0, 1:0, 2:0, 3:1, 4:1, 5:1, 
  6:2, 7:2, 8:2, 9:3, 10:3, 11:3}).astype(int)

data["YearsSinceRemodel"] = data["YrSold"] - data["YearRemodAdd"]
#简化一些关于质量的特征

# Simplifications of existing features into bad/average/good.
data["SimplOverallQual"] = data.OverallQual.replace(
{1 : 1, 2 : 1, 3 : 1, 4 : 2, 5 : 2, 6 : 2, 7 : 3, 8 : 3, 9 : 3, 10 : 3})
data["SimplOverallCond"] = data.OverallCond.replace(
{1 : 1, 2 : 1, 3 : 1, 4 : 2, 5 : 2, 6 : 2, 7 : 3, 8 : 3, 9 : 3, 10 : 3})
data["SimplPoolQC"] = data.PoolQC.replace(
{1 : 1, 2 : 1, 3 : 2, 4 : 2})
data["SimplGarageCond"] = data.GarageCond.replace(
{1 : 1, 2 : 1, 3 : 1, 4 : 2, 5 : 2})
data["SimplGarageQual"] = data.GarageQual.replace(
{1 : 1, 2 : 1, 3 : 1, 4 : 2, 5 : 2})
data["SimplFireplaceQu"] = data.FireplaceQu.replace(
{1 : 1, 2 : 1, 3 : 1, 4 : 2, 5 : 2})
data["SimplFireplaceQu"] = data.FireplaceQu.replace(
{1 : 1, 2 : 1, 3 : 1, 4 : 2, 5 : 2})
data["SimplFunctional"] = data.Functional.replace(
{1 : 1, 2 : 1, 3 : 2, 4 : 2, 5 : 3, 6 : 3, 7 : 3, 8 : 4})
data["SimplKitchenQual"] = data.KitchenQual.replace(
{1 : 1, 2 : 1, 3 : 1, 4 : 2, 5 : 2})
data["SimplHeatingQC"] = data.HeatingQC.replace(
{1 : 1, 2 : 1, 3 : 1, 4 : 2, 5 : 2})
data["SimplBsmtFinType1"] = data.BsmtFinType1.replace(
{1 : 1, 2 : 1, 3 : 1, 4 : 2, 5 : 2, 6 : 2})
data["SimplBsmtFinType2"] = data.BsmtFinType2.replace(
{1 : 1, 2 : 1, 3 : 1, 4 : 2, 5 : 2, 6 : 2})
data["SimplBsmtCond"] = data.BsmtCond.replace(
{1 : 1, 2 : 1, 3 : 1, 4 : 2, 5 : 2})
data["SimplBsmtQual"] = data.BsmtQual.replace(
{1 : 1, 2 : 1, 3 : 1, 4 : 2, 5 : 2})
data["SimplExterCond"] = data.ExterCond.replace(
{1 : 1, 2 : 1, 3 : 1, 4 : 2, 5 : 2})
data["SimplExterQual"] = data.ExterQual.replace(
{1 : 1, 2 : 1, 3 : 1, 4 : 2, 5 : 2})

In [15]:
#处理‘neighborhood’特征（有点乱），根据groupby以后的均值来离散化
# Bin by neighborhood (a little arbitrarily). Values were computed by: 
# train_df["SalePrice"].groupby(train_df["Neighborhood"]).median().sort_values()
neighborhood_map = {
        "MeadowV" : 0,  #  88000
        "IDOTRR" : 1,   # 103000
        "BrDale" : 1,   # 106000
        "OldTown" : 1,  # 119000
        "Edwards" : 1,  # 119500
        "BrkSide" : 1,  # 124300
        "Sawyer" : 1,   # 135000
        "Blueste" : 1,  # 137500
        "SWISU" : 2,    # 139500
        "NAmes" : 2,    # 140000
        "NPkVill" : 2,  # 146000
        "Mitchel" : 2,  # 153500
        "SawyerW" : 2,  # 179900
        "Gilbert" : 2,  # 181000
        "NWAmes" : 2,   # 182900
        "Blmngtn" : 2,  # 191000
        "CollgCr" : 2,  # 197200
        "ClearCr" : 3,  # 200250
        "Crawfor" : 3,  # 200624
        "Veenker" : 3,  # 218000
        "Somerst" : 3,  # 225500
        "Timber" : 3,   # 228475
        "StoneBr" : 4,  # 278000
        "NoRidge" : 4,  # 290000
        "NridgHt" : 4,  # 315000
    }
data["NeighborhoodBin"] = data["Neighborhood"].map(neighborhood_map)

- 3、对于float的缺失值，用中位数代替，剩余的int和obj缺失值，用上一行的值代替（要改用众数的值代替，可提高准确率）

In [16]:
#剩下的异常值不多了，而且集中在个位数的客户身上，直接用均值，字符型的用上一行的值代替
data_miss = data.isnull().apply(lambda x: x.sum(), axis=0)

data_miss = pd.DataFrame({'Index':data_miss.index,'Values':data_miss.values})
var=list()
for i,j in data_miss.iterrows():
    if data_miss.iloc[i,1] > 0:
        var.append(data_miss.iloc[i,0])

        
for i in var:
    if i != 'SalePrice' :
        if  data[i].dtype == 'float64':
            data[i] = data[i].apply(lambda x: data[i].median() if pd.isnull(x) else x) #中位数值替代Na和负数
            
        else:
            data[i].fillna(method='bfill',inplace=True)#用上一行的值代替

In [17]:
#最后查看缺省值
data_miss = data.isnull().apply(lambda x: x.sum(), axis=0)

data_miss = pd.DataFrame({'Index':data_miss.index,'Values':data_miss.values})
var=list()
for i,j in data_miss.iterrows():
    if data_miss.iloc[i,1] > 0:
        var.append(data_miss.iloc[i,0])


- 4、时间序列的重构，异常值用均值代替

In [18]:
#车库建成时间改为车库使用年限
#起建时间改为使用年限
#重建时间拆为：是否有重建、重建至今年限
#处理年份及异常值
#车库使用年份
data['GarageYr'] = map(lambda x,y:y-x ,data['GarageYrBlt'],data['YrSold'])
data['GarageYr'] = data['GarageYr'].apply(lambda x: data['GarageYr'].median() if pd.isnull(x) or x < 0 else x) #中位数值替代Na和负数


#房屋年份
data['BuiltYear'] = map(lambda x,y:y-x ,data['YearBuilt'],data['YrSold'])
data['BuiltYear'] = data['BuiltYear'].apply(lambda x: data['BuiltYear'].median() if pd.isnull(x) or x < 0 else x) #中位数值替代Na和负数


#翻修距今时间
data['RemodAddYear'] = map(lambda x,y:y-x ,data['YearRemodAdd'],data['YrSold'])
data['RemodAddYear'] = data['RemodAddYear'].apply(lambda x: data['RemodAddYear'].median() if pd.isnull(x) or x < 0 else x) #中位数值替代Na和负数




#LotFrontage、MasVnrArea也顺手用中值替换了

data['LotFrontage'] = data['LotFrontage'].apply(lambda x: data['LotFrontage'].median() if pd.isnull(x) else x) #中位数值替代Na和负数
data['MasVnrArea'] = data['MasVnrArea'].apply(lambda x: data['MasVnrArea'].median() if pd.isnull(x) else x) #中位数值替代Na和负数


#丢了处理过的字段
data.drop(['GarageYrBlt','YrSold','YearBuilt','MoSold','YearRemodAdd'],axis=1,inplace=True) #丢了


# 三、特征工程<br><br>

- 1、关于质量等的字符型特征，统一换成数值型

- 2、关于质量等的特征，统一换成数值型

# 四、处理其他特征，便于模型运行。增加或合并<br><br>


- 1、分布有较大偏差的列，尝试删除

#有某项大于1的，删除
df = pd.DataFrame()

for i in data.columns:
    if i != 'SalePrice':
#        if data[i].dtype == 'int64' or data[i].dtype == 'float64':
        df[i] = data[i]

var=list()
var_col = list()
for i in df.columns:
    var.append(i)

for v in var:
#    print '\n\n%s这一列特征的不同取值，共有%s个缺失值" \n'% (v,data[v].isnull().sum())
    if ((data[v].value_counts().values*1.0)/data[v].value_counts().sum() > 1).any():
        var_col.append(v)
data = data.drop(var_col,axis=1)

- 2、int64或float64，进行log变换，丢弃拟合程度高的列

#选择cat类数据，新建数据表df进行处理
df = pd.DataFrame()

for i in data.columns:
    if i != 'SalePrice':
        if data[i].dtype == 'int64' or data[i].dtype == 'float64':
            df[i] = data[i]

df_int = pd.DataFrame()
for i in df.columns:
    if df[i].dtype == 'float64' or df[i].dtype == 'int64' :
        df[i] = df[i].apply(lambda x:int(x)+1)

# TODO：为DataFrame创建一个副本，用'drop'函数丢弃一些指定的特征
var_DropLabel=list()
for i in df.columns:
    var_targets = None
    var_features = None
    var_targets = df[i]
    var_features = df.drop(i, axis = 1)


# TODO：使用给定的特征作为目标，将数据分割成训练集和测试集    
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(var_features, var_targets, test_size=0.25, random_state=0)

# TODO：创建一个决策树回归器并在训练集上训练它    
    clf = tree.DecisionTreeClassifier(random_state=0)
    clf=clf.fit(X_train,y_train)

# TODO：输出在测试集上的预测得分
    

    y_pred = clf.predict(X_test)
    score = None
    score = r2_score( y_test,y_pred)
    var_DropLabel.append([i,score])

var_DropLabel.sort(key=lambda x:x[1],reverse=True) 
ff=list()
for i in var_DropLabel:
    if i[1] > 0.7:
        ff.append(i[0])

print var_DropLabel

# 丢弃拟合度高的特征列
data = data.drop(ff,axis=1)


- 3、值较少、无缺失的int64和float64转为str，所有str进行标签编码

#特征小于4的，转为str
df = pd.DataFrame()

for i in data.columns:
    if i != 'SalePrice':
        if data[i].dtype == 'int64' or data[i].dtype == 'float64':
            df[i] = data[i]

var=list()
var_col1 = list()
var_col2 = list()
for i in df.columns:
    var.append(i)

for v in var:
    print v,data[v].value_counts().index.value_counts().sum()
    if data[v].value_counts().index.value_counts().sum() < 17:
        if v != 'Id':
            var_col1.append(v)
    else:
        if v != 'Id':
            var_col2.append(v)



for u in var_col1:
    data[u]=data[u].astype(str)


for u in var_col2:
    data[u]=data[u].astype(float)
        
#data.apply(lambda x:str(x) for x in data[var_col1].values)
#[x*1.0 for x in data[var_col2].values]

- 4、丢弃异常值多的行

# 对于每一个特征，找到值异常高或者是异常低的数据点
aa=list()
bb=list()

df = pd.DataFrame()
for i in data.columns:
    if i != 'SalePrice':
        if data[i].dtype == 'int64' or data[i].dtype == 'float64':
            df[i] = data[i]

for i in df.columns:
    if df[i].dtype == 'float64' or df[i].dtype == 'int64' :
        df[i] = df[i].apply(lambda x:int(x)+1)

log_data = list()
log_data = np.log(df)
for feature in log_data.keys():
    
    # TODO：计算给定特征的Q1（数据的25th分位点）
    Q1 = np.percentile(log_data[feature],25)
    
    # TODO：计算给定特征的Q3（数据的75th分位点）
    Q3 = np.percentile(log_data[feature],75)
    
    # TODO：使用四分位范围计算异常阶（1.5倍的四分位距）
    step = 1.5 * (Q3 - Q1)
    
    # 显示异常点
#    print "Data points considered outliers for the feature '{}':".format(feature)
    aa = log_data[~((log_data[feature] >= Q1 - step) & (log_data[feature] <= Q3 + step))].index
    bb.append(aa.values)



cc=list()
dd=list()
ff=list()
for i in bb:
#    print i
    for j in i:
        cc.append(j)
    
for item in cc:
    dd.append([item,cc.count(item)])

    
#去除重复值
ee = []
for i in dd:
    if not i in ee:
        ee.append(i)

#排序并显示
ee.sort(key=lambda x:x[1],reverse=True) 

#找到训练集中，异常点大于1个
for i in ee:
    if i[0] < 1460:
        if i[1] > 1:
            ff.append(i[0])

print len(ff)
print ff

data = data.drop(ff)


#当n=0，0.17	
#当n=1，0.14939	
#当n=2，0.15174	
#所以最优参数为1个异常点

# 四、标准化数值型，OneHot离散型

In [19]:
df = pd.DataFrame()
numeric_features=list()
for i in data.columns:
    if i != 'SalePrice' and i != 'Id':
        if data[i].dtype == 'int64'or data[i].dtype == 'float64' :
            df[i] = data[i]
            numeric_features.append(i)
# Transform the skewed numeric features by taking log(feature + 1).
# This will make the features more normal.
from scipy.stats import skew

skewed = df[numeric_features].apply(lambda x: skew(x.dropna().astype(float)))
skewed = skewed[skewed > 0.75]
skewed = skewed.index


df[skewed] = np.log1p(df[skewed])


# Additional processing: scale the data.   
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df[numeric_features])

scaled = scaler.transform(df[numeric_features])
for i, col in enumerate(numeric_features):
    df[col] = scaled[:, i]
    
    
for i in df.columns:
    data[i]=df[i]



In [20]:
data = pd.get_dummies(data)

# 五、清洗完的数据重新分开

In [21]:
#区分训练集和测试集

train = data.loc[data['source_train']==1]
test = data.loc[data['source_test']==1]

train.drop(['source_train','source_test'],axis=1,inplace=True)
test.drop(['source_train','source_test','SalePrice'],axis=1,inplace=True)


train.to_csv('train_modified.csv',index=False)
test.to_csv('test_modified.csv',index=False)

/Users/mahui/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/mahui/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
